In [1]:
from dotenv import load_dotenv
import os, pickle, asyncio
load_dotenv(".env")
OPENAI_KEY = os.environ.get("OPENAI_KEY")
from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

In [73]:
file = open("instructions.txt", "r")
instructions = file.read()
file = open("test.txt", "r")
abstracts = file.read()

In [ ]:
#Preprocessing

#list = abstracts.split("\n\n\n") #split into publications
#list1 = list[0].split("\n\n")
#print(list1[0]) #get identifier 
#print(max(list1, key=len)) #get abstract 


In [44]:
def get_category (instructions, abstracts):

   response = client.chat.completions.create(
   model="gpt-4-1106-preview" #"gpt-3.5-turbo-1106",
   #temperature=0.5,
  # response_format={ "type": "json_object" },
   messages=[
        {"role": "system", "content": instructions},
        {"role": "user", "content": abstracts}
    ]
   )
   return response.choices[0].message.content



In [74]:
with open("output.txt", "w") as file:
	content = get_category (instructions, abstracts)
	file.write(content)
	file.close()